In [1]:
import numpy as np

np.random.seed(0)
n = 100
a = np.random.normal(size=n).astype(np.float32)
b = np.random.normal(size=n).astype(np.float32)
c = a + b

In [2]:
def vector_add(a, b, c):
    for i in range(n):
        c[i] = a[i] + b[i]

d = np.empty(shape=n, dtype=np.float32)
vector_add(a, b, d)
np.testing.assert_array_equal(c, d)

In [3]:
# Save to the d2ltvm package.
def get_abc(shape, constructor=None):
    """Return random a, b and empty c with the same shape.
    """
    np.random.seed(0)
    a = np.random.normal(size=shape).astype(np.float32)
    b = np.random.normal(size=shape).astype(np.float32)
    c = np.empty_like(a)
    if constructor:
        a, b, c = [constructor(x) for x in (a, b, c)]
    return a, b, c

In [4]:
import tvm
from tvm import te # te stands for tensor expression

# Save to the d2ltvm package.
def vector_add(n):
    """TVM expression for vector add"""
    A = te.placeholder((n,), name='a')
    B = te.placeholder((n,), name='b')
    C = te.compute(A.shape, lambda i: A[i] + B[i], name='c')
    return A, B, C

A, B, C = vector_add(n)
type(A), type(C)

(tvm.te.tensor.Tensor, tvm.te.tensor.Tensor)

In [5]:
type(A.op), type(C.op)

(tvm.te.tensor.PlaceholderOp, tvm.te.tensor.ComputeOp)

In [6]:
s = te.create_schedule(C.op)

In [7]:
type(s), type(s[C])

(tvm.te.schedule.Schedule, tvm.te.schedule.Stage)

In [8]:
tvm.lower(s, [A, B, C], simple_mode=True)

#[version = "0.0.5"]
@main = primfn(a_1: handle, b_1: handle, c_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [100], []),
             b: Buffer(b_2: Pointer(float32), float32, [100], []),
             c: Buffer(c_2: Pointer(float32), float32, [100], [])}
  buffer_map = {a_1: a, b_1: b, c_1: c}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [100], []), b_1: b_3: Buffer(b_2, float32, [100], []), c_1: c_3: Buffer(c_2, float32, [100], [])} {
  for (i: int32, 0, 100) {
    c[i] = (a[i] + b[i])
  }
}

#[metadata]
{
  "root": 1, 
  "nodes": [
    {
      "type_key": ""
    }, 
    {
      "type_key": "Map", 
      "keys": [
        "IntImm"
      ], 
      "data": [2]
    }, 
    {
      "type_key": "Array", 
      "data": [3, 4]
    }, 
    {
      "type_key": "IntImm", 
      "attrs": {
        "dtype": "bool", 
        "span": "0", 
        "value": "1"
      }
 

In [9]:
mod = tvm.build(s, [A, B, C])
type(mod)

tvm.driver.build_module.OperatorModule

In [10]:
x = np.ones(2)
y = tvm.nd.array(x)
type(y), y.asnumpy()

(tvm.runtime.ndarray.NDArray, array([1., 1.]))

In [11]:
a, b, c = get_abc(100, tvm.nd.array)

In [16]:
mod(a, b, c)
np.testing.assert_array_equal(a.asnumpy() + b.asnumpy(), c.asnumpy())

## Saving and Loading a Module

In [17]:
mod_fname = 'vector-add.tar'
mod.export_library(mod_fname)

In [18]:
loaded_mod = tvm.runtime.load_module(mod_fname)

In [21]:
a, b, c = get_abc(100, tvm.nd.array)
loaded_mod(a, b, c)
np.testing.assert_array_equal(a.asnumpy() + b.asnumpy(), c.asnumpy())